In [3]:
import pandas as pd
import numpy as np
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout

In [4]:
emotion_df = pd.read_csv('emotion_features.csv', index_col=0)
emotion_df

,0,1,2,3,4,5,6,7,8,9,...,173,174,175,176,177,178,179,180,181,labels
0,0.274579,0.560966,0.521340,0.539612,0.584370,0.566328,0.590016,0.542824,0.542918,0.595936,...,-5.963057,-3.072044,-5.011403,-5.808564,-4.776329,-1.666140,-6.362252,-5.209133,-1.626034,angry
1,0.416943,0.780868,0.788520,0.810166,0.835733,0.828571,0.739328,0.689585,0.714463,0.779047,...,-3.392732,-2.344621,-3.281808,-2.752456,-1.234239,-1.442518,-2.517537,-2.682515,-1.980154,angry
2,0.224011,0.632741,0.648564,0.645445,0.590219,0.481998,0.450006,0.496442,0.509890,0.603403,...,-0.533864,-7.908329,-3.536236,-5.696254,-2.378804,-7.909769,2.263980,-1.307509,0.180049,angry
3,0.216470,0.588893,0.532681,0.540489,0.593834,0.576432,0.591428,0.558195,0.569552,0.616729,...,-6.407280,-4.029168,-5.764251,-6.581980,-4.459784,-1.555337,-6.031616,-5.524695,-1.640235,angry
4,0.223300,0.554488,0.512918,0.482543,0.464256,0.425962,0.413757,0.410311,0.429979,0.477790,...,-8.284494,1.015495,-6.648326,-6.796526,-3.946552,-6.189187,-2.909742,-4.325276,-4.083507,fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46147,0.060750,0.636769,0.585038,0.652742,0.616112,0.717962,0.669722,0.610102,0.653336,0.713311,...,-7.802677,9.156364,-10.524547,11.523617,-21.769642,11.141151,-14.523054,5.984777,-5.345934,neutral
46148,0.041161,0.751996,0.618682,0.534846,0.539487,0.630493,0.528285,0.520108,0.672241,0.719681,...,-6.086152,4.849677,-15.761593,9.697728,-16.421173,6.792037,-13.683490,5.655865,-10.450261,neutral
46149,0.300101,0.890704,0.812318,0.779347,0.788816,0.798159,0.654105,0.659761,0.782625,0.764981,...,-3.452307,-3.107889,-2.995228,-2.873970,-2.793679,-4.259409,-3.905076,-2.975534,-4.176648,neutral
46150,0.045441,0.546909,0.739436,0.626243,0.529209,0.527069,0.618814,0.524002,0.501906,0.658899,...,3.004317,-5.297758,-8.082297,3.791116,-12.532305,4.468070,-12.664186,7.091371,-15.475475,neutral


In [5]:
def extract_features_for_model(emotion_features_df):
    labels = emotion_features_df['labels']
    label_mapping = {
        'angry': 0,
        'fear': 1,
        'disgust': 2,
        'sad': 3,
        'happy': 4,
        'neutral': 5
    }
    
    numerical_labels = labels.map(label_mapping)
    emotion_features_df['labels'] = numerical_labels
    
    x = emotion_features_df.iloc[: ,:-1].values
    y = emotion_features_df['labels'].values
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, shuffle=True)
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_train = np.expand_dims(x_train, axis=2)
    x_test = np.expand_dims(x_test, axis=2)
    joblib.dump(scaler, 'scaler.bin', compress=True)
    
    return x_train, x_test, y_train, y_test

In [6]:
def create_cnn_model(input_shape, filters, kernel_size, dropout):
    model = Sequential()
    
    model.add(Conv1D(filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters * 2, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters * 4, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    
    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(6, activation='softmax'))
    
    model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])
    
    return model

In [7]:
x_train, x_test, y_train, y_test = extract_features_for_model(emotion_df)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((36921, 182, 1), (36921,), (9231, 182, 1), (9231,))

In [8]:
ser_cnn_model = create_cnn_model((182,1), 96, 5, 0.4)
ser_cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 178, 96)           576       
                                                                 
 max_pooling1d (MaxPooling1  (None, 89, 96)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 85, 192)           92352     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 42, 192)           0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 38, 384)           369024    
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 19, 384)           0

In [9]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
ser_cnn_model_history = ser_cnn_model.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_test, y_test), callbacks=[rlrp])

Epoch 1/20
1154/1154 [==============================] - 30s 26ms/step - loss: 1.2935 - accuracy: 0.4740 - val_loss: 1.1138 - val_accuracy: 0.5542 - lr: 0.0010
Epoch 2/20
1154/1154 [==============================] - 32s 28ms/step - loss: 1.0936 - accuracy: 0.5617 - val_loss: 1.1032 - val_accuracy: 0.5516 - lr: 0.0010
Epoch 3/20
1154/1154 [==============================] - 34s 29ms/step - loss: 0.9996 - accuracy: 0.6005 - val_loss: 1.0065 - val_accuracy: 0.5966 - lr: 0.0010
Epoch 4/20
1154/1154 [==============================] - 35s 31ms/step - loss: 0.9221 - accuracy: 0.6339 - val_loss: 0.9375 - val_accuracy: 0.6265 - lr: 0.0010
Epoch 5/20
1154/1154 [==============================] - 35s 30ms/step - loss: 0.8432 - accuracy: 0.6680 - val_loss: 0.9531 - val_accuracy: 0.6333 - lr: 0.0010
Epoch 6/20
1154/1154 [==============================] - 34s 30ms/step - loss: 0.7714 - accuracy: 0.6970 - val_loss: 0.9102 - val_accuracy: 0.6492 - lr: 0.0010
Epoch 7/20
1154/1154 [========================

In [10]:
print("Accuracy of model on test data : ", ser_cnn_model.evaluate(x_test, y_test)[1] * 100, "%")

289/289 [==============================] - 3s 11ms/step - loss: 1.2623 - accuracy: 0.7118
Accuracy of model on test data :  71.18405103683472 %


In [11]:
ser_cnn_model.save('new-tuned-ser-cnn-model.h5')

/opt/anaconda3/envs/tf_env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
